## These are a series of notebooks presented at the 2019 Esri Developer Summit Conference

They highlight configuring a Workforce Project by adding users, assignment types, integrations, and assignments using various ways. The notebooks should be followed in order. By substituting the credentials, layers, and project, with your own information these workflows should be easily replicated.

These notebooks require the ArcGIS API for Python version 1.6 or higher as well as shapely or arcpy installed.

In the blocks below, there is some code that can be used to throughout the notebooks (if necessary).

### Reset Project
The following block can be used to reset the Workforce Project to the original state.

In [ ]:
import arcgis
from arcgis.apps import workforce

gis = arcgis.gis.GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)

project.assignments_item.layers[0].delete_features(where="1=1")
project.dispatchers.batch_delete(project.dispatchers.search("userId <> 'workforce_scripts'"))
project.workers.batch_delete(project.workers.search())
project.assignment_types.batch_delete(project.assignment_types.search())

### Build SQLite Database and Table
For one of the notebooks, assignments are read from a SQLite database. The following block creates the database using sqlite3 and pandas.

In [ ]:
import sqlite3
from datetime import datetime
import pandas as pd

df = pd.DataFrame(
[
    [1, datetime(2018, 7, 12), "200 S Palm Canyon Dr, Palm Springs, CA 92262", 1, "Sidewalk Repair", "Completed", "The sidewalk needs to be fixed.", "Done"],
    [2, datetime(2018, 7, 13), "500 S Palm Canyon Dr, Palm Springs, CA 92264", 1, "Sidewalk Repair", "Completed", "The sidewalk is uneven due to tree roots.", "Finished"],
    [3, datetime(2018, 7, 14), "700 S Calle Palo Fierro, Palm Springs, CA 92264", 2, "Sidewalk Repair", "Backlog", "The sidewalk is very uneven.", None],
    [4, datetime(2018, 7, 15), "100 S Sunrise Way, Palm Springs, CA 92262", 2, "Sidewalk Repair", "Backlog", "Please fix the sidewalk near the intersection", None],
    [5, datetime(2018, 7, 16), "300 S Farrell Dr, Palm Springs, CA 92262", 3, "Sidewalk Repair", "Backlog", "Fix my side walk", None],
    [6, datetime(2018, 7, 12), "900 N Palm Canyon Dr, Palm Springs, CA 92262", 3, "Sidewalk Repair", "Backlog", "Fix the side walk in front of my shop", None],
],
columns=["id", "due_date", "address", "priority", "type", "status", "description", "notes"])

connection = sqlite3.connect("work_orders")
df.to_sql("work_orders", connection, if_exists="replace")
df